In [0]:
!pip install --upgrade pip

  Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


Instalando o gdal manualmente

In [0]:
%sh
#!/bin/bash
sudo apt-get update
# Isso instala o gdal na máquina
sudo apt-get install -y gdal-bin libgdal-dev
sudo apt-get install -y libproj-dev

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Fetched 114 kB in 1s (157 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (3.0.4+dfsg-1build3).
libgdal-dev is already the newest version (3.0.4+dfsg-1build3).
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
libproj-dev is already the newest version (6.3.1-1).
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.


os arquivos precisam estar com os nomes corretos

In [0]:
%sh
cd /usr/lib/python3/dist-packages/osgeo
mv _gdal.cpython-38-x86_64-linux-gnu.so _gdal.so
mv _gdalconst.cpython-38-x86_64-linux-gnu.so _gdalconst.so
mv _ogr.cpython-38-x86_64-linux-gnu.so _ogr.so
mv _osr.cpython-38-x86_64-linux-gnu.so _osr.so
mv _gdal_array.cpython-38-x86_64-linux-gnu.so _gdal_array.so
ls /usr/lib/python3/dist-packages/osgeo

mv: cannot stat '_gdal.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_gdalconst.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_ogr.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_osr.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_gdal_array.cpython-38-x86_64-linux-gnu.so': No such file or directory
__init__.py
__pycache__
_gdal.so
_gdal_array.so
_gdalconst.so
_gnm.cpython-38-x86_64-linux-gnu.so
_ogr.so
_osr.so
gdal.py
gdal_array.py
gdalconst.py
gdalnumeric.py
gnm.py
ogr.py
osr.py


Bibliotecas

In [0]:
from pyspark.sql import SparkSession
import os

In [0]:
!pip install pyspark raster2xyz

  Using cached pyspark-3.5.1-py2.py3-none-any.whl
  Using cached raster2xyz-0.1.3-py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)


Iniciando sessão spark

In [0]:
spark=SparkSession.builder \
    .master('local[*]') \
    .getOrCreate()

Função para avaliar o benchmark

In [0]:
def benchmark(arquivo):
    df = spark.read.format("csv").option("header", True).load(arquivo)
    df.write.format("noop").mode("overwrite").save()

In [0]:
def partitions(arquivo):
  df = spark.read.format("csv").option("header", True).load(arquivo)
  print(f"Número de partições -> {df.rdd.getNumPartitions()}")

#Precipitação

##Precipitação de 10 minutos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_prec.zip
!unzip -o wc2.1_10m_prec.zip

--2024-05-04 22:33:51--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_prec.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_prec.zip [following]
--2024-05-04 22:33:51--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_prec.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7258539 (6.9M) [application/zip]
Saving to: ‘wc2.1_10m_prec.zip’

wc2.1_10m_prec.zip  100%[===================>]   6.92M  19.8MB/s    in 0.3s    

2024-05-04 22:33:51 (19.8 MB/s) - ‘wc2.1_10m_prec.zip’ saved [7258539/7258539]

Archive:  wc2.1_10m_prec.zip
  inflat

In [0]:
!raster2xyz wc2.1_10m_prec_01.tif wc2.1_10m_prec_01.csv

[2024-05-04 10:33:54 - INFO] - Getting geotransform and data...
[2024-05-04 10:33:54 - INFO] - Getting XYZ data...
[2024-05-04 10:33:54 - INFO] - Getting geotransformed coordinates...
[2024-05-04 10:33:54 - INFO] - Building XYZ data...
[2024-05-04 10:34:02 - INFO] - New XYZ (csv file) created...


Os arquivos são salvos no cluster então é necessário movê-los para o dbfs

In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_10m_prec_01.tif", "dbfs:/FileStore/tables/wc2.1_10m_prec_01.tif")

Out[12]: True

In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_10m_prec_01.csv", "dbfs:/FileStore/tables/wc2.1_10m_prec_01.csv")

Out[13]: True

Testando o benchmark

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_10m_prec_01.csv'

In [0]:
partitions(arquivo)

Número de partições -> 8


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 6.23 ms, sys: 374 µs, total: 6.6 ms
Wall time: 4.87 s
CPU times: user 5.97 ms, sys: 2.35 ms, total: 8.33 ms
Wall time: 4.73 s
CPU times: user 4.74 ms, sys: 4.03 ms, total: 8.77 ms
Wall time: 4.7 s
CPU times: user 9.03 ms, sys: 0 ns, total: 9.03 ms
Wall time: 5.05 s
CPU times: user 8.62 ms, sys: 0 ns, total: 8.62 ms
Wall time: 4.97 s
CPU times: user 12.1 ms, sys: 118 µs, total: 12.2 ms
Wall time: 4.83 s
CPU times: user 8.5 ms, sys: 119 µs, total: 8.62 ms
Wall time: 5.05 s
CPU times: user 9.47 ms, sys: 271 µs, total: 9.74 ms
Wall time: 5.05 s
CPU times: user 6.1 ms, sys: 2.41 ms, total: 8.51 ms
Wall time: 4.99 s
CPU times: user 6.76 ms, sys: 1.08 ms, total: 7.84 ms
Wall time: 5.1 s
CPU times: user 7.79 ms, sys: 622 µs, total: 8.41 ms
Wall time: 5.19 s
CPU times: user 3.64 ms, sys: 4.48 ms, total: 8.12 ms
Wall time: 5.09 s
CPU times: user 8.95 ms, sys: 0 ns, total: 8.95 ms
Wall time: 4.83 s
CPU times: user 4.43 ms, sys: 3.79 ms, total: 8.22 ms
Wall time: 5.02 s
CPU times: 

##Precipitação 5 minutos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_prec.zip
!unzip -o wc2.1_5m_prec.zip

--2024-05-04 22:36:48--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_prec.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_prec.zip [following]
--2024-05-04 22:36:49--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_prec.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22696275 (22M) [application/zip]
Saving to: ‘wc2.1_5m_prec.zip’

wc2.1_5m_prec.zip   100%[===================>]  21.64M  30.3MB/s    in 0.7s    

2024-05-04 22:36:49 (30.3 MB/s) - ‘wc2.1_5m_prec.zip’ saved [22696275/22696275]

Archive:  wc2.1_5m_prec.zip
  inflating:

In [0]:
!raster2xyz wc2.1_5m_prec_01.tif wc2.1_5m_prec_01.csv

[2024-05-04 10:36:52 - INFO] - Getting geotransform and data...
[2024-05-04 10:36:52 - INFO] - Getting XYZ data...
[2024-05-04 10:36:52 - INFO] - Getting geotransformed coordinates...
[2024-05-04 10:36:53 - INFO] - Building XYZ data...
[2024-05-04 10:37:26 - INFO] - New XYZ (csv file) created...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_5m_prec_01.csv", "dbfs:/FileStore/tables/wc2.1_5m_prec_01.csv")

Out[19]: True

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_5m_prec_01.csv'

In [0]:
partitions(arquivo)

Número de partições -> 8


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 12.2 ms, sys: 4.61 ms, total: 16.8 ms
Wall time: 15.5 s
CPU times: user 9.93 ms, sys: 4.24 ms, total: 14.2 ms
Wall time: 15.1 s
CPU times: user 12.1 ms, sys: 702 µs, total: 12.8 ms
Wall time: 14.9 s
CPU times: user 11.9 ms, sys: 677 µs, total: 12.6 ms
Wall time: 16.5 s
CPU times: user 11.1 ms, sys: 2.66 ms, total: 13.8 ms
Wall time: 15.9 s
CPU times: user 11.5 ms, sys: 1.25 ms, total: 12.8 ms
Wall time: 15.1 s
CPU times: user 11.2 ms, sys: 1.18 ms, total: 12.3 ms
Wall time: 15.7 s
CPU times: user 11.3 ms, sys: 3.19 ms, total: 14.5 ms
Wall time: 15.8 s
CPU times: user 8.71 ms, sys: 5.32 ms, total: 14 ms
Wall time: 16.1 s
CPU times: user 8.68 ms, sys: 4.69 ms, total: 13.4 ms
Wall time: 16.2 s
CPU times: user 3.34 ms, sys: 11 ms, total: 14.3 ms
Wall time: 16.5 s
CPU times: user 11.4 ms, sys: 4.64 ms, total: 16 ms
Wall time: 15.9 s
CPU times: user 9.45 ms, sys: 4.85 ms, total: 14.3 ms
Wall time: 15.7 s
CPU times: user 8.92 ms, sys: 4.77 ms, total: 13.7 ms
Wall time: 16.2 s


##Precipitação 2.5

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_prec.zip
!unzip -o wc2.1_2.5m_prec.zip

--2024-05-04 22:46:11--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_prec.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_prec.zip [following]
--2024-05-04 22:46:11--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_prec.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71780834 (68M) [application/zip]
Saving to: ‘wc2.1_2.5m_prec.zip’

wc2.1_2.5m_prec.zip 100%[===================>]  68.46M  34.2MB/s    in 2.0s    

2024-05-04 22:46:13 (34.2 MB/s) - ‘wc2.1_2.5m_prec.zip’ saved [71780834/71780834]

Archive:  wc2.1_2.5m_prec.zip


In [0]:
!raster2xyz wc2.1_2.5m_prec_01.tif wc2.1_2.5m_prec_01.csv

[2024-05-04 10:46:16 - INFO] - Getting geotransform and data...
[2024-05-04 10:46:16 - INFO] - Getting XYZ data...
[2024-05-04 10:46:18 - INFO] - Getting geotransformed coordinates...
[2024-05-04 10:46:20 - INFO] - Building XYZ data...
[2024-05-04 10:48:35 - INFO] - New XYZ (csv file) created...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_2.5m_prec_01.csv", "dbfs:/FileStore/tables/wc2.1_2.5m_prec_01.csv")

Out[25]: True

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_2.5m_prec_01.csv'


In [0]:
partitions(arquivo)

Número de partições -> 11


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 19.2 ms, sys: 12.8 ms, total: 32 ms
Wall time: 1min
CPU times: user 20 ms, sys: 11.9 ms, total: 32 ms
Wall time: 57.6 s
CPU times: user 19.5 ms, sys: 13.2 ms, total: 32.7 ms
Wall time: 58.2 s
CPU times: user 28.1 ms, sys: 8.59 ms, total: 36.7 ms
Wall time: 57.1 s
CPU times: user 21.3 ms, sys: 9.42 ms, total: 30.7 ms
Wall time: 57 s
CPU times: user 21.4 ms, sys: 14.8 ms, total: 36.2 ms
Wall time: 58.9 s
CPU times: user 12.8 ms, sys: 19.2 ms, total: 32 ms
Wall time: 58.3 s
CPU times: user 15 ms, sys: 16.9 ms, total: 31.9 ms
Wall time: 57.2 s
CPU times: user 23.5 ms, sys: 10.5 ms, total: 34 ms
Wall time: 1min
CPU times: user 22.9 ms, sys: 11.7 ms, total: 34.6 ms
Wall time: 59.8 s
CPU times: user 23.5 ms, sys: 12.5 ms, total: 36 ms
Wall time: 1min 2s
CPU times: user 32.8 ms, sys: 2.36 ms, total: 35.1 ms
Wall time: 59.2 s
CPU times: user 14.6 ms, sys: 18.9 ms, total: 33.5 ms
Wall time: 59.2 s
CPU times: user 18.3 ms, sys: 13.1 ms, total: 31.4 ms
Wall time: 57.8 s
CPU times: 

##Precipitação de 30 segundos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_prec.zip
!unzip -o wc2.1_30s_prec.zip

--2024-05-04 23:21:24--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_prec.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_prec.zip [following]
--2024-05-04 23:21:25--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_prec.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025401472 (978M) [application/zip]
Saving to: ‘wc2.1_30s_prec.zip’

wc2.1_30s_prec.zip  100%[===================>] 977.90M  36.4MB/s    in 27s     

2024-05-04 23:21:52 (36.5 MB/s) - ‘wc2.1_30s_prec.zip’ saved [1025401472/1025401472]

Archive:  wc2.1_30s_prec.zip

In [0]:
#arquivo mt grande databricks n tankou 
!raster2xyz wc2.1_30s_prec_01.tif wc2.1_30s_prec_01.csv

[2024-05-04 11:22:05 - INFO] - Getting geotransform and data...
[2024-05-04 11:22:12 - INFO] - Getting XYZ data...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_30s_prec_01.csv", "dbfs:/FileStore/tables/wc2.1_30s_prec_01.csv")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-167082039869024>:1
----> 1 dbutils.fs.cp("file:/databricks/driver/wc2.1_30s_prec_01.csv", "dbfs:/FileStore/tables/wc2.1_30s_prec_01.csv")

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o652.cp.
: java.io.FileNotFoundException: File file:/databricks/driver/wc2.1_30s_prec_01.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoo

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_30s_prec_01.csv'

In [0]:
for i in range(1,33):
 %time benchmark(arquivo)